<a href="https://colab.research.google.com/github/haticebaydemir/PotansiyelMusteriGetirisiHesaplama/blob/main/PotansiyelMusteriGetirisiHesaplama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gezinomi Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

In [4]:
import numpy as np
import pandas as pd
from warnings import filterwarnings

# GÖREV 1:
Aşağıdaki soruları yanıtlayınız.
 Soru 1: miuul_gezinomi.xlsx dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

 Soru 2: Kaç unique şehir vardır? Frekansları nedir?

 Soru 3: Kaç unique Concept vardır?

 Soru 4: Hangi Concept'dan kaçar tane satış gerçekleşmiş?

 Soru 5: Şehirlere göre satışlardan toplam ne kadar kazanılmış?

 Soru 6: Concept türlerine göre göre ne kadar kazanılmış?

 Soru 7: Şehirlere göre PRICE ortalamaları nedir?

 Soru 8: Conceptlere  göre PRICE ortalamaları nedir?

 Soru 9: Şehir-Concept kırılımında PRICE ortalamaları nedir?



## Soru 1: miuul_gezinomi.xlsx dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

In [11]:
import zipfile

# ZIP dosyasının yolu
file_path = '/content/drive/MyDrive/gezinomi_dataset/gezinomi.zip'

# ZIP dosyasını aç ve içindeki Excel dosyasını oku
with zipfile.ZipFile(file_path, 'r') as z:
    with z.open('gezinomi_data/miuul_gezinomi.xlsx') as f:
        df = pd.read_excel(f)  # Dosyayı oku
        print(df.head())  # Verilerin ilk birkaç satırını yazdır

   SaleId   SaleDate CheckInDate       Price     ConceptName SaleCityName  \
0  415122 2022-12-03  2022-12-03   79.304029    Herşey Dahil      Antalya   
1  415103 2022-12-03  2022-12-03   45.970696  Yarım Pansiyon      Antalya   
2  404034 2022-09-12  2022-09-13   77.838828    Herşey Dahil      Antalya   
3  415094 2022-12-03  2022-12-10  222.710623  Yarım Pansiyon        İzmir   
4  414951 2022-12-01  2022-12-03  140.476190  Yarım Pansiyon        İzmir   

     CInDay  SaleCheckInDayDiff Seasons  
0  Saturday                   0     Low  
1  Saturday                   0     Low  
2   Tuesday                   1    High  
3  Saturday                   7     Low  
4  Saturday                   2     Low  


In [12]:
# Veri çerçevesinin yapısı
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB
None


In [13]:
# İlk 5 satırı görüntüle
print(df.head())

   SaleId   SaleDate CheckInDate       Price     ConceptName SaleCityName  \
0  415122 2022-12-03  2022-12-03   79.304029    Herşey Dahil      Antalya   
1  415103 2022-12-03  2022-12-03   45.970696  Yarım Pansiyon      Antalya   
2  404034 2022-09-12  2022-09-13   77.838828    Herşey Dahil      Antalya   
3  415094 2022-12-03  2022-12-10  222.710623  Yarım Pansiyon        İzmir   
4  414951 2022-12-01  2022-12-03  140.476190  Yarım Pansiyon        İzmir   

     CInDay  SaleCheckInDayDiff Seasons  
0  Saturday                   0     Low  
1  Saturday                   0     Low  
2   Tuesday                   1    High  
3  Saturday                   7     Low  
4  Saturday                   2     Low  


In [14]:
# Veri çerçevesinin boyutunu (satır, sütun) göster
print(f"Veri seti boyutu: {df.shape}")

Veri seti boyutu: (59164, 9)


In [17]:
# Sütun isimlerini görüntüle
print("Sütun isimleri:", df.columns.tolist())

Sütun isimleri: ['SaleId', 'SaleDate', 'CheckInDate', 'Price', 'ConceptName', 'SaleCityName', 'CInDay', 'SaleCheckInDayDiff', 'Seasons']


In [18]:
# Sayısal sütunlar için temel istatistikler
print(df.describe())

              SaleId                       SaleDate  \
count   59164.000000                          59164   
mean   302675.795433  2021-02-11 17:13:11.751740928   
min     51707.000000            2016-01-02 00:00:00   
25%    274038.500000            2020-08-19 00:00:00   
50%    321607.000000            2021-07-28 00:00:00   
75%    367373.000000            2022-05-10 00:00:00   
max    415122.000000            2022-12-03 00:00:00   
std     87943.222683                            NaN   

                         CheckInDate         Price  SaleCheckInDayDiff  
count                          59164  59151.000000        59164.000000  
mean   2021-03-15 11:23:01.732134400     61.345057           31.840173  
min              2007-07-20 00:00:00      0.000000            0.000000  
25%              2020-08-24 00:00:00     35.343035            2.000000  
50%              2021-08-16 00:00:00     54.261294           10.000000  
75%              2022-07-04 00:00:00     77.962578           34.00

## Soru 2: Kaç unique şehir vardır? Frekansları nedir?

In [24]:
# Şehir sütununu tanımlayın
sehir_sutun_adi = 'SaleCityName'

# Benzersiz şehirlerin sayısını bulma
unique_sehirler = df[sehir_sutun_adi].unique()
print(f"Benzersiz şehir sayısı: {len(unique_sehirler)}")

# Şehir frekanslarını bulma
sehir_frekanslari = df[sehir_sutun_adi].value_counts()
print("\nŞehir Frekansları:")
print(sehir_frekanslari)

# Soru 2: Kaç unique şehir vardır? Frekansları nedir?
#print(df["SaleCityName"].nunique())
#print(df["SaleCityName"].value_counts())

Benzersiz şehir sayısı: 6

Şehir Frekansları:
SaleCityName
Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: count, dtype: int64


## Soru 3: Kaç unique Concept vardır?

In [30]:
# Şehir sütununu tanımlayın
concept_sutun_adi = 'ConceptName'

# Benzersiz şehirlerin sayısını bulma
unique_concepts = df[concept_sutun_adi].unique()
print(f"Unique concept sayısı: {len(unique_concepts)}")

# Concept frekanslarını bulma
concept_frekanslari = df[concept_sutun_adi].value_counts()
print("\nConcept Frekansları:")
print(concept_frekanslari)

# Soru 3: Kaç unique Concept vardır?
#df["ConceptName"].nunique()

Unique concept sayısı: 3

Concept Frekansları:
ConceptName
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: count, dtype: int64


## Soru 4: Hangi Concept'dan kaçar tane satış gerçekleşmiş?

In [32]:
# Yine frekans bakacağız.
# 'ConceptName' sütunundaki her bir konseptin satış sayısını bulma
concept_sales_counts = df['ConceptName'].value_counts()

# Sonuçları yazdırma
print("Her bir konseptin satış sayıları:")
print(concept_sales_counts)

Her bir konseptin satış sayıları:
ConceptName
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: count, dtype: int64


## Soru 5: Şehirlere göre satışlardan toplam ne kadar kazanılmış?

In [33]:
# Şehir ve satış miktarlarını içeren sütun isimlerini belirleyin
sehir_sutun_adi = 'SaleCityName'  # Şehir sütun adı
satis_miktari_sutun_adi = 'Price'  # Satış miktarı sütun adı (bunu doğru isimle değiştirin)

# Şehirlere göre toplam kazancı hesaplama
toplam_kazanc = df.groupby(sehir_sutun_adi)[satis_miktari_sutun_adi].sum()

# Sonuçları yazdırma
print("Şehirlere göre toplam kazanç:")
print(toplam_kazanc)

Şehirlere göre toplam kazanç:
SaleCityName
Antalya    2.041911e+06
Aydın      5.732960e+05
Diğer      1.545723e+05
Girne      2.706503e+04
Muğla      6.658422e+05
İzmir      1.659348e+05
Name: Price, dtype: float64


In [35]:
df.groupby("SaleCityName").agg({"Price": "sum"})

,Price
SaleCityName,
Antalya,2.041911e+06
Aydın,5.732960e+05
Diğer,1.545723e+05
Girne,2.706503e+04
Muğla,6.658422e+05
İzmir,1.659348e+05


## Soru 6: Concept türlerine göre göre ne kadar kazanılmış?

In [36]:
df.groupby("ConceptName").agg({"Price": "sum"})

,Price
ConceptName,
Herşey Dahil,3.332911e+06
Oda + Kahvaltı,1.213084e+05
Yarım Pansiyon,1.744024e+05


## Soru 7: Şehirlere göre PRICE ortalamaları nedir?

In [37]:
df.groupby("SaleCityName").agg({"Price": "mean"})

,Price
SaleCityName,
Antalya,64.521474
Aydın,53.855896
Diğer,47.707497
Girne,59.483579
Muğla,62.461746
İzmir,66.267905


In [39]:
df.groupby(by=['SaleCityName']).agg({"Price": "mean"})

,Price
SaleCityName,
Antalya,64.521474
Aydın,53.855896
Diğer,47.707497
Girne,59.483579
Muğla,62.461746
İzmir,66.267905


## Soru 8: Conceptlere göre PRICE ortalamaları nedir?

In [38]:
df.groupby("ConceptName").agg({"Price": "mean"})

,Price
ConceptName,
Herşey Dahil,62.672260
Oda + Kahvaltı,50.252010
Yarım Pansiyon,49.030742


## Soru 9: Şehir-Concept kırılımında PRICE ortalamaları nedir?

In [40]:
df.groupby(by=["SaleCityName", 'ConceptName']).agg({"Price": "mean"})

Price
SaleCityName ConceptName              
Antalya      Herşey Dahil    64.519028
             Oda + Kahvaltı  63.504816
             Yarım Pansiyon  67.190613
Aydın        Herşey Dahil    53.995467
             Oda + Kahvaltı  34.458787
             Yarım Pansiyon  30.016543
Diğer        Herşey Dahil    84.771093
             Oda + Kahvaltı  37.599082
             Yarım Pansiyon  42.113440
Girne        Herşey Dahil    97.680970
             Oda + Kahvaltı  39.775536
             Yarım Pansiyon  53.247838
Muğla        Herşey Dahil    63.020053
             Oda + Kahvaltı  59.037763
             Yarım Pansiyon  45.120924
İzmir        Herşey Dahil    74.701519
             Oda + Kahvaltı  41.320342
             Yarım Pansiyon  59.610053

# GÖREV 2:
satis_checkin_day_diff değişkenini EB_Score adında yeni bir kategorik değişkene çeviriniz.

In [41]:
bins = [-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]

df["EB_Score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels=labels)
df.head(50).to_excel("eb_scorew.xlsx", index=False)


# GÖREV 3:
Şehir,Concept, [EB_Score,Sezon,CInday] kırılımında ücret ortalamalarına ve frekanslarına bakınız


In [ ]:
# Şehir-Concept-EB Score kırılımında ücret ortalamaları
# Şehir-Concept-Sezon kırılımında ücret ortalamaları
# Şehir-Concept-CInday kırılımında ücret ortalamaları

In [42]:
# Şehir-Concept-EB Score kırılımında ücret ortalamaları
df.groupby(by=["SaleCityName", 'ConceptName', 'EB_Score']).agg({"Price": "mean"})

Price
SaleCityName ConceptName    EB_Score                     
Antalya      Herşey Dahil   Last Minuters       62.752916
                            Potential Planners  64.903700
                            Planners            67.879960
                            Early Bookers       66.494885
             Oda + Kahvaltı Last Minuters       65.353740
...                                                   ...
İzmir        Oda + Kahvaltı Early Bookers       66.928111
             Yarım Pansiyon Last Minuters       57.788126
                            Potential Planners  64.804043
                            Planners            60.507887
                            Early Bookers       65.905583

[72 rows x 1 columns]

In [43]:
df.groupby(by=["SaleCityName", 'ConceptName', "EB_Score" ]).agg({"Price": ["mean", "count"]})

Price       
                                                     mean  count
SaleCityName ConceptName    EB_Score                            
Antalya      Herşey Dahil   Last Minuters       62.752916  14148
                            Potential Planners  64.903700   8874
                            Planners            67.879960   4490
                            Early Bookers       66.494885   3281
             Oda + Kahvaltı Last Minuters       65.353740    503
...                                                   ...    ...
İzmir        Oda + Kahvaltı Early Bookers       66.928111      3
             Yarım Pansiyon Last Minuters       57.788126    636
                            Potential Planners  64.804043    198
                            Planners            60.507887     47
                            Early Bookers       65.905583     14

[72 rows x 2 columns]

In [44]:
# Şehir-Concept-Sezon kırılımında ücret ortalamaları
df.groupby(by=["SaleCityName", 'ConceptName', 'Seasons']).agg({"Price": "mean"})

Price
SaleCityName ConceptName    Seasons            
Antalya      Herşey Dahil   High      64.920065
                            Low       61.552427
             Oda + Kahvaltı High      66.267969
                            Low       60.666730
             Yarım Pansiyon High      73.259029
                            Low       62.001678
Aydın        Herşey Dahil   High      54.946560
                            Low       33.680699
             Oda + Kahvaltı High      30.387530
                            Low       44.451874
             Yarım Pansiyon High      32.626152
                            Low       25.271799
Diğer        Herşey Dahil   High      83.787273
                            Low       87.310882
             Oda + Kahvaltı High      34.839543
                            Low       39.959569
             Yarım Pansiyon High      39.731916
                            Low       42.953978
Girne        Herşey Dahil   High     103.935448
                            Low       90.935944
             Oda + Kahvaltı High      39.479630
                            Low       40.043067
             Yarım Pansiyon High      58.383668
                            Low       48.578903
Muğla        Herşey Dahil   High      63.384036
                            Low       48.304086
             Oda + Kahvaltı High      61.264691
                            Low       37.774192
             Yarım Pansiyon High      45.823698
                            Low       32.681812
İzmir        Herşey Dahil   High      74.748363
                            Low       74.308287
             Oda + Kahvaltı High      54.141797
                            Low       33.555235
             Yarım Pansiyon High      87.657303
                            Low       55.350626

In [45]:
df.groupby(by=["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": ["mean", "count"]})

Price       
                                           mean  count
SaleCityName ConceptName    Seasons                   
Antalya      Herşey Dahil   High      64.920065  27126
                            Low       61.552427   3667
             Oda + Kahvaltı High      66.267969    303
                            Low       60.666730    295
             Yarım Pansiyon High      73.259029    118
                            Low       62.001678    138
Aydın        Herşey Dahil   High      54.946560  10103
                            Low       33.680699    473
             Oda + Kahvaltı High      30.387530     27
                            Low       44.451874     11
             Yarım Pansiyon High      32.626152     20
                            Low       25.271799     11
Diğer        Herşey Dahil   High      83.787273    364
                            Low       87.310882    141
             Oda + Kahvaltı High      34.839543    349
                            Low       39.959569    408
             Yarım Pansiyon High      39.731916    516
                            Low       42.953978   1462
Girne        Herşey Dahil   High     103.935448     55
                            Low       90.935944     51
             Oda + Kahvaltı High      39.479630     66
                            Low       40.043067     73
             Yarım Pansiyon High      58.383668    100
                            Low       48.578903    110
Muğla        Herşey Dahil   High      63.384036   9582
                            Low       48.304086    237
             Oda + Kahvaltı High      61.264691    592
                            Low       37.774192     62
             Yarım Pansiyon High      45.823698    177
                            Low       32.681812     10
İzmir        Herşey Dahil   High      74.748363   1234
                            Low       74.308287    147
             Oda + Kahvaltı High      54.141797     86
                            Low       33.555235    142
             Yarım Pansiyon High      87.657303    118
                            Low       55.350626    777

In [46]:
# Şehir-Concept-CInday kırılımında ücret ortalamaları
df.groupby(by=["SaleCityName", 'ConceptName', 'CInDay']).agg({"Price": "mean"})

Price
SaleCityName ConceptName    CInDay              
Antalya      Herşey Dahil   Friday     62.658940
                            Monday     63.259143
                            Saturday   64.415526
                            Sunday     65.849177
                            Thursday   62.892734
...                                          ...
İzmir        Yarım Pansiyon Saturday   52.504725
                            Sunday     98.934697
                            Thursday   52.484894
                            Tuesday    56.141381
                            Wednesday  54.064392

[126 rows x 1 columns]

In [47]:
df.groupby(by=["SaleCityName", "ConceptName", "CInDay"]).agg({"Price": ["mean", "count"]})

Price      
                                            mean count
SaleCityName ConceptName    CInDay                    
Antalya      Herşey Dahil   Friday     62.658940  4136
                            Monday     63.259143  6831
                            Saturday   64.415526  4741
                            Sunday     65.849177  3818
                            Thursday   62.892734  3898
...                                          ...   ...
İzmir        Yarım Pansiyon Saturday   52.504725   315
                            Sunday     98.934697    68
                            Thursday   52.484894    91
                            Tuesday    56.141381    73
                            Wednesday  54.064392    86

[126 rows x 2 columns]

# GÖREV 4:
City-Concept-Season kırılımın çıktısını PRICE'a göre sıralayınız.

In [55]:
# Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
# Çıktıyı agg_df olarak kaydediniz.

In [53]:
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "mean"}).sort_values("Price", ascending=False)
agg_df.head(20)
#Bu kod, SaleCityName, ConceptName, ve Seasons sütunlarını kullanarak veriyi gruplandırır ve her bir grubun Price (fiyat) değerinin ortalamasını alarak en yüksek fiyat ortalamalarına göre sıralar.

Price
SaleCityName ConceptName    Seasons            
Girne        Herşey Dahil   High     103.935448
                            Low       90.935944
İzmir        Yarım Pansiyon High      87.657303
Diğer        Herşey Dahil   Low       87.310882
                            High      83.787273
İzmir        Herşey Dahil   High      74.748363
                            Low       74.308287
Antalya      Yarım Pansiyon High      73.259029
             Oda + Kahvaltı High      66.267969
             Herşey Dahil   High      64.920065
Muğla        Herşey Dahil   High      63.384036
Antalya      Yarım Pansiyon Low       62.001678
             Herşey Dahil   Low       61.552427
Muğla        Oda + Kahvaltı High      61.264691
Antalya      Oda + Kahvaltı Low       60.666730
Girne        Yarım Pansiyon High      58.383668
İzmir        Yarım Pansiyon Low       55.350626
Aydın        Herşey Dahil   High      54.946560
İzmir        Oda + Kahvaltı High      54.141797
Girne        Yarım Pansiyon Low       48.578903

# GÖREV 5:
Indekste yer alan isimleri değişken ismine çeviriniz.

In [56]:
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
# Bu isimleri değişken isimlerine çeviriniz.
# İpucu: reset_index()

In [57]:
agg_df.reset_index(inplace=True)

agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935448
1,Girne,Herşey Dahil,Low,90.935944
2,İzmir,Yarım Pansiyon,High,87.657303
3,Diğer,Herşey Dahil,Low,87.310882
4,Diğer,Herşey Dahil,High,83.787273


# GÖREV 6:
Yeni level based satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.


In [58]:
# sales_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
agg_df['sales_level_based'] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].agg(lambda x: '_'.join(x).upper(), axis=1)

In [59]:
agg_df

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH
5,İzmir,Herşey Dahil,High,74.748363,İZMIR_HERŞEY DAHIL_HIGH
6,İzmir,Herşey Dahil,Low,74.308287,İZMIR_HERŞEY DAHIL_LOW
7,Antalya,Yarım Pansiyon,High,73.259029,ANTALYA_YARIM PANSIYON_HIGH
8,Antalya,Oda + Kahvaltı,High,66.267969,ANTALYA_ODA + KAHVALTI_HIGH
9,Antalya,Herşey Dahil,High,64.920065,ANTALYA_HERŞEY DAHIL_HIGH


# GÖREV 7:
Personaları segmentlere ayırınız.



In [60]:
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz
# segmentleri betimleyiniz
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})

Price                        
              mean         max         sum
SEGMENT                                   
D        33.366288   39.479630  300.296592
C        44.887654   54.141797  403.988887
B        60.274498   64.920065  542.470478
A        82.467833  103.935448  742.210498

# GÖREV 8:
Oluşan son df'i price değişkenine göre sıralayınız.

In [61]:
agg_df.sort_values(by="Price")

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
35,Aydın,Yarım Pansiyon,Low,25.271799,AYDIN_YARIM PANSIYON_LOW,D
34,Aydın,Oda + Kahvaltı,High,30.387530,AYDIN_ODA + KAHVALTI_HIGH,D
33,Aydın,Yarım Pansiyon,High,32.626152,AYDIN_YARIM PANSIYON_HIGH,D
32,Muğla,Yarım Pansiyon,Low,32.681812,MUĞLA_YARIM PANSIYON_LOW,D
31,İzmir,Oda + Kahvaltı,Low,33.555235,İZMIR_ODA + KAHVALTI_LOW,D
30,Aydın,Herşey Dahil,Low,33.680699,AYDIN_HERŞEY DAHIL_LOW,D
29,Diğer,Oda + Kahvaltı,High,34.839543,DIĞER_ODA + KAHVALTI_HIGH,D
28,Muğla,Oda + Kahvaltı,Low,37.774192,MUĞLA_ODA + KAHVALTI_LOW,D
27,Girne,Oda + Kahvaltı,High,39.479630,GIRNE_ODA + KAHVALTI_HIGH,D
26,Diğer,Yarım Pansiyon,High,39.731916,DIĞER_YARIM PANSIYON_HIGH,C


# "ANTALYA_HERŞEY DAHIL_HIGH" hangi segmenttedir ve ne kadar ücret beklenmektedir?


In [67]:
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"] == new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
9,Antalya,Herşey Dahil,High,64.920065,ANTALYA_HERŞEY DAHIL_HIGH,B
